In [ ]:
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D
import keras
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
input_shape=(28,28,1)

print('x_train.shape=',x_train.shape)

#normalisation
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train/=255
x_test/=255


#convet class vectors to binary class matrices: 3 becomes [0,0,0,1,0,0,0,0,0,0]
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)
print(y_test)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
x_train.shape= (60000, 28, 28, 1)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from tensorflow.keras import Sequential
from keras.layers.convolutional import Convolution2D
def model_cnn():
  model = Sequential()
  model.add(Convolution2D(64, 3, 3,padding='valid',input_shape=(28, 28,1)))
  #model.add(Convolution2D(32, 3, 3,padding='valid',input_shape=(28, 28,1)))

# now: model.output_shape == (None, 64, 32, 32)
  model.add(Flatten())
  model.add(Dense(128,activation='relu'))

  model.add(Dense(10,activation='softmax'))
  model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  return model

  '''
  l_in = tf.keras.layers.Input(shape=(h, w, c,))
  l_act = l_in

  # Simple Conv model
  # l_act = tf.keras.layers.BatchNormalization()(l_act)
  l_act = tf.keras.layers.Conv2D(16, 3, activation='relu')(l_act)
  l_act = tf.keras.layers.MaxPool2D(2)(l_act)

  # l_act = tf.keras.layers.BatchNormalization()(l_act)
  l_act = tf.keras.layers.Conv2D(24, 3, activation='relu')(l_act)
  l_act = tf.keras.layers.ZeroPadding2D(((0, 1), (0, 1)))(l_act)
  l_act = tf.keras.layers.MaxPool2D(2)(l_act)

  # l_act = tf.keras.layers.BatchNormalization()(l_act)
  l_act = tf.keras.layers.Conv2D(32, 3, activation='relu')(l_act)

  l_act = tf.keras.layers.GlobalMaxPooling2D()(l_act)
  l_act = tf.keras.layers.Dense(nbc, activation='softmax')(l_act)

  l_out = l_act
  return tf.keras.models.Model(l_in, l_out)
  '''


In [ ]:
#correction
from tensorflow.keras.layers import Dropout,Conv2D,MaxPooling2D
import matplotlib.pyplot as plt
def model_gen():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  '''Correction prof'''
  def model_gen2():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dense(10, activation='softmax'))


In [ ]:
my_model=model_cnn()

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
my_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)





In [ ]:
#metrics
#traiter les données:


#sklearn.metrics.classification_report(y_test, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')
from sklearn.metrics import classification_report
import numpy as np
Y_test = np.argmax(y_test, axis=-1) # Convert one-hot to index
y_pred = my_model.predict_classes(x_test)
print(classification_report(Y_test, y_pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       0.02      0.01      0.01       980
           1       0.00      0.00      0.00      1135
           2       0.00      0.00      0.00      1032
           3       0.11      0.79      0.19      1010
           4       0.19      0.11      0.14       982
           5       0.20      0.02      0.04       892
           6       0.17      0.00      0.00       958
           7       0.06      0.06      0.06      1028
           8       0.00      0.00      0.00       974
           9       0.03      0.00      0.00      1009

    accuracy                           0.10     10000
   macro avg    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Correction
my_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

 

my_model.fit(x_train, y_train,
          batch_size=512,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

 

score = my_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
118/118 [==============================] - 1s 7ms/step - loss: 2.3018 - accuracy: 0.1107 - val_loss: 2.2940 - val_accuracy: 0.1192
Epoch 2/10
118/118 [==============================] - 1s 5ms/step - loss: 2.2862 - accuracy: 0.1385 - val_loss: 2.2774 - val_accuracy: 0.1541
Epoch 3/10
118/118 [==============================] - 1s 5ms/step - loss: 2.2696 - accuracy: 0.1827 - val_loss: 2.2600 - val_accuracy: 0.2055
Epoch 4/10
118/118 [==============================] - 1s 5ms/step - loss: 2.2521 - accuracy: 0.2331 - val_loss: 2.2416 - val_accuracy: 0.2572
Epoch 5/10
 78/118 [==================>...........] - ETA: 0s - loss: 2.2369 - accuracy: 0.2733

KeyboardInterrupt: ignored

In [ ]:
#with API fonctionnelle
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs,
name='mnist_model')

keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
model.compile(loss='sparse_categorical_crossentropy',
 optimizer=keras.optimizers.RMSprop(),
 metrics=['accuracy'])
history = model.fit(x_train, y_train,
 batch_size=64,
 epochs=5,
 validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


Epoch 1/5
750/750 [==============================] - 2s 3ms/step - loss: 0.3538 - accuracy: 0.8991 - val_loss: 0.1772 - val_accuracy: 0.9495
Epoch 2/5
750/750 [==============================] - 2s 3ms/step - loss: 0.1606 - accuracy: 0.9519 - val_loss: 0.1403 - val_accuracy: 0.9600
Epoch 3/5
750/750 [==============================] - 2s 3ms/step - loss: 0.1190 - accuracy: 0.9646 - val_loss: 0.1192 - val_accuracy: 0.9654
Epoch 4/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0951 - accuracy: 0.9714 - val_loss: 0.1112 - val_accuracy: 0.9676
Epoch 5/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0792 - accuracy: 0.9762 - val_loss: 0.1146 - val_accuracy: 0.9662
313/313 - 1s - loss: 0.1063 - accuracy: 0.9694
Test loss: 0.10628946870565414
Test accuracy: 0.9693999886512756


In [ ]:

#encoder
encoder_input = keras.Input(shape=(28, 28, 1), name='original_img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)
encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()




NameError: ignored

In [ ]:


#Décodeur

decoder_input = keras.Input(shape=(16,), name='encoded_img')
x = layers.Reshape((4, 4, 1))(decoder_input)#OK : 4*4=16
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)
decoder = keras.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()
autoencoder_input = keras.Input(shape=(28, 28, 1), name='img')
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name='autoencoder')
autoencoder.summary()


Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoded_img (InputLayer)     [(None, 16)]              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 1)           0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 6, 6, 16)          160       
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 8, 8, 32)          4640      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 26, 26, 16)        4624      
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 28, 28, 1)         145 

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
autoencoder.compile(loss='sparse_categorical_crossentropy',
 optimizer=keras.optimizers.RMSprop(),
 metrics=['accuracy'])
'''               
history=autoencoder.fit(x_train, x_train,
                         epochs=10,
                         shuffle=True,
                         validation_data=(x_test, x_test))
'''


from keras import losses
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


autoencoder.fit(x_train, x_train,
                         epochs=10,
                         shuffle=True,
                         validation_data=(x_test, x_test))




test_scores = autoencoder.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0215 - val_loss: 0.0211
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0212 - val_loss: 0.0206
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0210 - val_loss: 0.0208
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0209 - val_loss: 0.0203
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0207 - val_loss: 0.0204
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0205 - val_loss: 0.0201
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0204 - val_loss: 0.0199
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0203 - val_loss: 0.0199
Epoch 9/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0202 - val_loss: 0.0198
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.020

InvalidArgumentError: ignored

In [ ]:


inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)


model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

 

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

    
import tensorflow as tf
from tensorflow import keras
import numpy as np

from tensorflow.keras.datasets import mnist,fashion_mnist
from tensorflow.keras import layers
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)


model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')


model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(train_images, train_labels, epochs=5, batch_size=128)
 
    
(x_train, _), (x_test, _) = fashion_mnist.load_data()


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


print(x_train.shape)










NameError: ignored

In [ ]:


from keras import losses
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


autoencoder.fit(x_train, x_train,
                         epochs=10,
                         shuffle=True,
                         validation_data=(x_test, x_test))


Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0199 - val_loss: 0.0195
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0198 - val_loss: 0.0202
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0198 - val_loss: 0.0195
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0197 - val_loss: 0.0194
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0196 - val_loss: 0.0197
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0195 - val_loss: 0.0192
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0195 - val_loss: 0.0195
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0194 - val_loss: 0.0192
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0193 - val_loss: 0.0190
Epoch 10/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.019